In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import XLNetForSequenceClassification, XLNetTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Define your model and tokenizer
model_name = "xlnet-base-cased"
model = XLNetForSequenceClassification.from_pretrained(model_name)
tokenizer = XLNetTokenizer.from_pretrained(model_name)


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load dataset
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/politifact_cleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=1.0, random_state=42)


# Split the dataset into training, validation, and test sets
# First, split into training and temp sets (80% training, 20% temp)
train_df, temp_df = train_test_split(df_data, test_size=0.20, random_state=42)

# Then, split the temp set into validation and test sets (50% validation, 50% test)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally,reset the index of the DataFrames
train_df = train_df.reset_index(drop=True)
#validation_df = validation_df1.sample(frac=0.50, random_state=42)

validation_df = validation_df.reset_index(drop=True)

#test_df = test_df1.sample(frac=0.50, random_state=42)
test_df = test_df.reset_index(drop=True)

In [ ]:
df.groupby("label").describe()

Unnamed: 0                                                           \
           count   mean         std    min     25%    50%     75%     max   
label                                                                       
0          432.0  839.5  124.851912  624.0  731.75  839.5  947.25  1055.0   
1          624.0  311.5  180.277564    0.0  155.75  311.5  467.25   623.0   

      word count                                                   
           count       mean       std  min  25%   50%   75%   max  
label                                                              
0          432.0  11.701389  5.222315  1.0  9.0  11.0  14.0  38.0  
1          624.0   7.931090  4.665029  1.0  5.0   7.0  10.0  53.0

In [ ]:
train_df.groupby("label").describe()

Unnamed: 0                                                             \
           count     mean         std    min     25%    50%     75%     max   
label                                                                         
0          344.0  842.250  127.068042  624.0  728.75  848.0  954.25  1055.0   
1          500.0  316.502  177.314068    0.0  167.75  321.5  466.50   623.0   

      word count                                                  
           count      mean       std  min  25%   50%   75%   max  
label                                                             
0          344.0  11.56686  5.280599  1.0  9.0  11.0  14.0  38.0  
1          500.0   7.96000  4.706453  1.0  4.0   7.0  10.0  53.0

In [ ]:
validation_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  805.795455  104.407426  629.0  732.75  795.5  857.50   
1           62.0  307.677419  196.936852   39.0  113.25  297.5  499.25   

              word count                                                    
          max      count       mean       std  min  25%   50%    75%   max  
label                                                                       
0      1046.0       44.0  11.795455  5.428519  2.0  9.0  12.0  13.25  37.0  
1       622.0       62.0   8.161290  5.353698  1.0  5.0   7.5   9.00  31.0

In [ ]:
test_df.groupby("label").describe()

Unnamed: 0                                                        \
           count        mean         std    min     25%    50%     75%   
label                                                                    
0           44.0  851.704545  123.172516  625.0  758.75  862.0  951.25   
1           62.0  274.983871  185.405227    6.0  126.50  251.5  421.75   

              word count                                                     
          max      count       mean       std  min   25%   50%    75%   max  
label                                                                        
0      1050.0       44.0  12.659091  4.513253  5.0  10.0  12.0  14.25  30.0  
1       618.0       62.0   7.467742  3.481651  1.0   5.0   7.0  10.00  16.0

In [ ]:
df.head()

,Unnamed: 0,id,title,label,word count,SECTION_CLEANED
0,0,politifact14984,National Federation of Independent Business,1,5,National Federation Independent Business
1,1,politifact12944,comments in Fayetteville NC,1,4,comment Fayetteville NC
2,2,politifact333,"Romney makes pitch, hoping to close deal : Ele...",1,14,Romney make pitch hop close deal Elections The...
3,3,politifact4358,Democratic Leaders Say House Democrats Are Uni...,1,11,Democratic Leaders Say House Democrats Are Uni...
4,4,politifact779,"Budget of the United States Government, FY 2008",1,8,Budget United States Government FY 2008


In [ ]:
train_texts = list(train_df['SECTION_CLEANED'])
len(train_texts)

844

In [ ]:
train_labels = list(train_df['label'])
len(train_labels)

844

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [ ]:
# Tokenize the dataset
train_encodings = tokenizer(train_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
train_labels = torch.tensor(train_labels)

In [ ]:
train_dataset = CustomDataset(train_encodings, train_labels)

<ipython-input-14-91207c483471>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels)


In [ ]:
val_texts = list(validation_df['SECTION_CLEANED'])
len(val_texts)

106

In [ ]:
val_labels = list(validation_df['label'])
len(val_labels)

106

In [ ]:
# Tokenize the dataset
val_encodings = tokenizer(val_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
val_labels = torch.tensor(val_labels)

In [ ]:
# Define a custom dataset for validation
val_dataset = CustomDataset(val_encodings, val_labels)

<ipython-input-14-91207c483471>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./xlnet_fake_news_classification_model",
    evaluation_strategy="steps",
    num_train_epochs=7,
    save_steps=75,
    eval_steps=75,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    warmup_steps=992,
    weight_decay=0.001047076633830012,
    logging_dir="./logs",
    logging_steps=75,
)

In [ ]:
# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
75,0.242000,0.348049
150,0.201200,0.427583
225,0.170500,0.493049
300,0.130600,0.824518


TrainOutput(global_step=371, training_loss=0.1775764814926929, metrics={'train_runtime': 106.3096, 'train_samples_per_second': 55.574, 'train_steps_per_second': 3.49, 'total_flos': 214387989753168.0, 'train_loss': 0.1775764814926929, 'epoch': 7.0})

In [ ]:
# Save the model if needed
trainer.save_model("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/XLNet/politifact_nlr_7")

In [ ]:
results = trainer.evaluate(val_dataset)

In [ ]:
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.8155367970466614, 'eval_runtime': 0.4555, 'eval_samples_per_second': 232.716, 'eval_steps_per_second': 30.736, 'epoch': 7.0}


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Load the model for inference
loaded_model = XLNetForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/XLNet/politifact_nlr_7")


In [ ]:
# Make predictions on the validation set
with torch.no_grad():
    validation_outputs = loaded_model(**val_encodings)
    val_logits = validation_outputs.logits
    val_predicted_labels = torch.argmax(val_logits, dim=1)

In [ ]:
# Calculate evaluation metrics for the validation set
accuracy_val = accuracy_score(val_labels, val_predicted_labels)
precision_val = precision_score(val_labels, val_predicted_labels)
recall_val = recall_score(val_labels, val_predicted_labels)
f1_val = f1_score(val_labels, val_predicted_labels)

In [ ]:
print(f"Validation Accuracy: {accuracy_val*100:.4f}")
print(f"Validation Precision: {precision_val*100:.4f}")
print(f"Validation Recall: {recall_val*100:.4f}")
print(f"Validation F1 Score: {f1_val*100:.4f}")

Validation Accuracy: 88.6792
Validation Precision: 89.0625
Validation Recall: 91.9355
Validation F1 Score: 90.4762


In [ ]:
test_texts = list(test_df['SECTION_CLEANED'])
len(test_texts)

106

In [ ]:
test_labels = list(test_df['label'])
len(test_labels)

106

In [ ]:
test_encodings = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
test_labels = torch.tensor(test_labels)

In [ ]:
# Make predictions on the test set
with torch.no_grad():
    test_outputs = loaded_model(**test_encodings)
    test_logits = test_outputs.logits
    test_predicted_labels = torch.argmax(test_logits, dim=1)

In [ ]:
# Calculate evaluation metrics for the test set
test_accuracy = accuracy_score(test_labels, test_predicted_labels)
test_precision = precision_score(test_labels, test_predicted_labels)
test_recall = recall_score(test_labels, test_predicted_labels)
test_f1 = f1_score(test_labels, test_predicted_labels)

In [ ]:
print("\nTest Set Metrics:")
print(f"Test Accuracy: {test_accuracy*100:.4f}")
print(f"Test Precision: {test_precision*100:.4f}")
print(f"Test Recall: {test_recall*100:.4f}")
print(f"Test F1 Score: {test_f1*100:.4f}")


Test Set Metrics:
Test Accuracy: 86.7925
Test Precision: 88.7097
Test Recall: 88.7097
Test F1 Score: 88.7097
